In [2]:
import os
%cd ..
%pwd

e:\code\projects\endtoend


c:\Users\priya\anaconda3\envs\cnncancer\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'e:\\code\\projects\\endtoend'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path 
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentated: bool
    params_image_size: int

In [8]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> TrainingConfig:
        config = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path_join(self.config.data_ingestion.unzip_dir, 'data')
        create_directories([config.root_dir])

        return TrainingConfig(
            root_dir=Path(config.root_dir),
            trained_model_path=Path(config.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentated=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

In [12]:
from zipfile import ZipFile
import urllib.request as request
import tensorflow as tf


In [13]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.2
        )
    
        dataflow_kwargs = dict(
            target_size=(self.config.params_image_size[:-1]),
            batch_size=self.config.params_batch_size,
            class_mode='categorical'
        )

        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_gen = valid_datagen.flow_from_directory(
            self.config.training_data,
            subset='validation',
            **dataflow_kwargs
        )

        if self.config.params_is_augmentated:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    **datagenerator_kwargs
            )
        else: train_datagen = valid_datagen

        self.train_generator = train_datagen.flow_from_directory(
            self.config.training_data,
            subset='training',
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_gen.samples // self.valid_gen.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_gen,
            validation_steps=self.validation_steps
        )

        self.save_model(self.config.trained_model_path, self.model)

In [ ]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    training = Training(prepare_base_model_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e